In [ ]:
import numpy as np
import pandas as pd
from maxvolpy.maxvol import rect_maxvol, maxvol
import random
from itertools import chain, cycle, islice

In [ ]:
list(islice(cycle(range(3)), 5))

## Does it drop redundant features?

$ 2 $ classes. We are generating a sample. Randomly choose one, then generate features for it. Different classes have different distributions of features. We generate $ n $ samples this way, suppose they all have $ k $ features. Replicate each column $ r $ times, so that we have $ n $ samples each with $ rk $ columns now. Add noise to each column.

Invoke maxvol to select $ k $ columns. See how many duplicates it gets. The hypothesis we're checking is it selects very few duplicates.

In [ ]:
def build_dataset(
    n, class_1_prob, k, min_feature_std, max_feature_std, how_to_duplicate,
    num_duplicates, noise_level, random_seed
):
    np.random.seed(random_seed)
    random.seed(random_seed)
    
    # let all features of class 1 have mean 1
    # and let all features of class 2 have mean 3
    class_1_features_means = 2 * np.ones(k)
    class_2_features_means = 4 * np.ones(k)
    
    # now let's choose standard deviations for features
    # for each feature its standard deviation will be the same no matter class 1 or class 2
    features_stds = np.linspace(min_feature_std, max_feature_std, num=k)
    
    # determine how many samples will be of each class
    class_1_num_samples = np.random.binomial(n, class_1_prob)
    class_2_num_samples = n - class_1_num_samples
    
    # now let's actually generate the data
    class_1_dataset = features_stds * np.random.randn(class_1_num_samples, k) + class_1_features_means
    class_2_dataset = features_stds * np.random.randn(class_2_num_samples, k) + class_2_features_means
    
    # in the following array first class_1_num_samples rows contain objects of class 1
    # and all rows after those contain objects of class 2
    dataset = np.concatenate((class_1_dataset, class_2_dataset), axis=0)
    
    return (
        duplicate_and_add_noise(
            dataset, how_to_duplicate, num_duplicates,
            noise_level, min_feature_std, max_feature_std
        ),
        class_1_num_samples,
        class_2_num_samples
    )

In [ ]:
def duplicate_and_add_noise(
    dataset, how_to_duplicate, num_duplicates,
    noise_level, min_feature_std, max_feature_std
):
    k = dataset.shape[1]
    dataset = np.tile(dataset, num_duplicates)
    
    noise_std = noise_level * (min_feature_std + max_feature_std) / 2
    dataset += noise_std * np.random.randn(*dataset.shape)
    
    if how_to_duplicate == "same":
        return dataset
    elif how_to_duplicate == "transforms":
        # add random shifts
        shifts = 10 * dataset.mean(axis=0) * (np.random.rand(k*num_duplicates) - 0.5)
        dataset += shifts
        
        transform_functions = list(islice(
            cycle([
                lambda arr: arr,
                np.exp,
                lambda arr: np.sqrt(np.abs(arr)),
                lambda arr: np.power(arr, 2),
                lambda arr: np.power(arr, 3)
            ]),
            num_duplicates
        ))
        for transform, batch in zip(transform_functions, range(num_duplicates)):
            dataset[:, batch*k:(batch+1)*k] = transform(dataset[:, batch*k:(batch+1)*k])
        return dataset
    else:
        raise ValueError("Incorrect `how_to_duplicate` parameter")

In [ ]:
k = 200 # number of true features
num_duplicates = 5
dataset, class_1_num_samples, class_2_num_samples = build_dataset(
    n=10000,
    class_1_prob=0.75,
    k=k,
    min_feature_std=0.5,
    max_feature_std=1.5,
    how_to_duplicate="transforms",
    num_duplicates=num_duplicates,
    noise_level=0.2,
    random_seed=6666
)

In [ ]:
print(dataset[:class_1_num_samples, 0].mean())
print(dataset[class_1_num_samples:, 0].mean())

# difference should be approximately 2

In [ ]:
print(dataset[:class_1_num_samples, k].mean())
print(dataset[class_1_num_samples:, k].mean())

print(dataset[:class_1_num_samples, k].std())
print(dataset[class_1_num_samples:, k].std())

We see that for each $ i \in \{ 0, \dots, \text{ num_true_features} \} $ correlation of each column with number $ i + j \text{ num_true_features} $ is far from zero.

In [ ]:
i = 1
pd.DataFrame(dataset[:, range(i, i + (num_duplicates-1)*k + 1, k)]).corr()

Now we want to choose the best k-column submatrix using `rect_maxvol`.

First let's try simply choosing k random rows.

In [ ]:
normalized_dataset = (dataset - dataset.mean(axis=0)) / dataset.std(axis=0)

In [ ]:
normalized_dataset.mean(axis=0)
# all values should be close to 0

In [ ]:
normalized_dataset.std(axis=0)
# all values should be close to 1

In [ ]:
def choose_features(dataset, k, samples_choice):
    n = dataset.shape[0]
    if samples_choice == "random":
        samples_subset_indices = np.random.choice(n, size=k, replace=False)
    elif samples_choice == "rect_maxvol":
        raise ValueError("Not implemented yet")
    else:
        raise ValueError("Incorrect samples_choice parameter")
    features_subset_indices = rect_maxvol(dataset[samples_subset_indices, :].T, minK=k, maxK=k, tol=0.05)[0]
    return features_subset_indices

In [ ]:
def calculate_percentage_uniq_features(features_subset_indices):
    return len(np.unique(features_subset_indices % k)) / len(features_subset_indices)

In [ ]:
chosen_features_indices = choose_features(normalized_dataset, k=k, samples_choice="random")
#print(chosen_features_indices)
#print(chosen_features_indices % k)
print(calculate_percentage_uniq_features(chosen_features_indices)) # more is better